In [1]:
import csv
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver

In [8]:
driver = webdriver.Chrome('/Users/pavloshcherban/Desktop/drivers/chromedriver-2')

In [9]:
url = "https://www.amazon.com"
driver.get(url)

In [10]:
def get_url(search_term):
   """"Generate a url from search term"""
   template = 'https://www.amazon.com/s?k={}&s=date-desc-rank&qid=1621432665&ref=sr_st_date-desc-rank'
   search_term = search_term.replace(' ', '+')
   return template.format(search_term)

In [11]:
url = get_url('ultrawide monitor')
print(url)

https://www.amazon.com/s?k=ultrawide+monitor&s=date-desc-rank&qid=1621432665&ref=sr_st_date-desc-rank


In [12]:
driver.get(url)

In [13]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [14]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [15]:
len(results)

18

In [16]:
item = results[0]

In [17]:
atag = item.h2.a

In [18]:
description = atag.text.strip()

In [19]:
url = 'https://www.amazon.com' + atag.get('href')

In [20]:
price_parent = item.find('span', 'a-price')

In [21]:
price = price_parent.find('span', 'a-offscreen').text

In [22]:
rating = item.i.text

In [23]:
review_count = item.find('span', {'class': 'a-size-base'}).text

In [24]:
def extract_record(item):
    """Extract and return data from a single record"""

    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    try:
        # price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return

    try:
        #rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)

    return result

In [25]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [26]:
records[0]

('Deco Gear 35” Curved Gaming Ultrawide Monitor, 3440x1440, 120hz, 1ms MPRT, 21:9, 99% sRGB',
 '$409.99',
 '4.6 out of 5 stars',
 '724',
 'https://www.amazon.com/Deco-Gear-Ultrawide-Monitor-3440x1440/dp/B0BCTZJ38R/ref=sr_1_1?keywords=ultrawide+monitor&qid=1665155613&qu=eyJxc2MiOiI2LjU0IiwicXNhIjoiNi4zMCIsInFzcCI6IjUuNzMifQ%3D%3D&sr=8-1')

In [27]:
for row in records:
    print(row[1])

$409.99
$192.98
$279.97
$249.99
$229.99
$1,049.98
$938.99
$579.94
$1,299.98
$469.99
$597.48
$1,978.99
$499.99
$349.99
$429.99
$449.99
$279.97
$129.99


In [28]:
def get_url(search_term):
   """"Generate a url from search term"""
   template = 'https://www.amazon.com/s?k={}&s=date-desc-rank&qid=1621432665&ref=sr_st_date-desc-rank'
   search_term = search_term.replace(' ', '+')
   
   # add term query to url
   url = template.format(search_term)
   
   # add page query placeholder
   url += '&page=' '{}'

   return url

In [31]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_term):
   """"Generate a url from search term"""
   template = 'https://www.amazon.com/s?k={}&s=date-desc-rank&qid=1621432665&ref=sr_st_date-desc-rank'
   search_term = search_term.replace(' ', '+')
   
   # add term query to url
   url = template.format(search_term)
   
   # add page query placeholder
   url += '&page=' '{}'

   return url

def extract_record(item):
    """Extract and return data from a single record"""

    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    try:
        # price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return

    try:
        #rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)

    return result

def main(search_term):
    """Run main program routine"""
    # startup the webdriver
    driver = webdriver.Chrome('/Users/pavloshcherban/Desktop/drivers/chromedriver-2')

    records = []
    url = get_url(search_term)

    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})

        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)

    driver.close()

    # save data to csv file
    with open('results1(3).csv', 'a+', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)             

In [32]:
main('ultrawide monitor')